In [1]:
from pprint import pprint
import pandas as pd
import nltk
import regex as re
import tools as t

Получение данных о скилах во всех вакансиях

In [2]:
full = pd.read_json('deduplicated.json')['key_skills']
skills = []
for element in full:
    if len(element) > 0:
        skills = skills + element
print(len(skills))

105375


Навыки, которые не нужно разделять

In [3]:
not_to_split = ['a/b', 'ci/cd', 'pl/sql', 'pl/pg', 'tcp/ip', 'ui/ux', 'ux/ui', 'а/б', 'а/в', 'в/из']

Разделение перечислений на отдельные навыки

In [4]:
skills = t.split_skills(skills, not_to_split)

In [5]:
dff = pd.DataFrame(skills)
dff['orig'] = dff[0]
dff = dff.drop(columns = 0)

Удаление нерелевантных символов

In [511]:
dff['clean'] = dff['orig'].apply(lambda x: t.clean_text(x, not_to_split))

Токенизация

In [8]:
from nltk import word_tokenize

In [9]:
#dff_1c['tokenize'] = dff_1c['clean'].apply(lambda x: word_tokenize(x, language='russian'))
dff['tokenize'] = dff['clean'].apply(lambda x: word_tokenize(x, language='russian'))

Лемматизация

In [10]:
#dff_1c['lemmatize'] = dff_1c['tokenize'].apply(lambda x: t.lemmatization(x))
dff['lemmatize'] = dff['tokenize'].apply(lambda x: t.lemmatization(x))

Скачивание стоп-слов для русского языка и добавление новых стоп-слов

In [11]:
from nltk.corpus import stopwords
sw_to_add = ['хорошее', 'знание', 'знать', 'опыт', 'работы', 'организация', 'работать', 'ms', 'условия', 'умение', 'навык']
sw_to_add = t.lemmatization(sw_to_add)
stopwords = stopwords.words('russian')
stopwords = stopwords + sw_to_add

Удаление стоп-слов

In [12]:
#dff_1c['lemmatize'] = dff_1c['lemmatize'].apply(lambda x: t.remove_stop_words(x, stopwords))
dff['lemmatize'] = dff['lemmatize'].apply(lambda x: t.remove_stop_words(x, stopwords))

Составление n-грамм

In [13]:
from nltk import ngrams

In [14]:
#dff_1c['bigrams'] = dff_1c['lemmatize'].apply(lambda x: list(ngrams(x, 2)))
dff['bigrams'] = dff['lemmatize'].apply(lambda x: list(ngrams(x, 2)))

#dff_1c['trigrams'] = dff_1c['lemmatize'].apply(lambda x: list(ngrams(x, 3)))
dff['trigrams'] = dff['lemmatize'].apply(lambda x: list(ngrams(x, 3)))

#dff_1c['fourgrams'] = dff_1c['lemmatize'].apply(lambda x: list(ngrams(x, 4)))
dff['fourgrams'] = dff['lemmatize'].apply(lambda x: list(ngrams(x, 4)))

Получение частоты n-грамм, для которых частота >=50

In [568]:
#most_common_1c = t.most_frequent_ngrams(dff_1c['lemmatize'], 50)
most_common = t.most_frequent_ngrams(dff['lemmatize'], 1)

#most_common_bigrams_1c = t.most_frequent_ngrams(dff_1c['bigrams'], 50)
most_common_bigrams = t.most_frequent_ngrams(dff['bigrams'], 1)

#most_common_trigrams_1c = t.most_frequent_ngrams(dff_1c['trigrams'], 50)
most_common_trigrams = t.most_frequent_ngrams(dff['trigrams'], 1)

#most_common_fourgrams_1c = t.most_frequent_ngrams(dff_1c['fourgrams'], 50)
most_common_fourgrams = t.most_frequent_ngrams(dff['fourgrams'], 1)

Сохранение полученных n-грамм

In [22]:
#df_ngrams_1c = pd.concat([pd.DataFrame({'unigrams': most_common_1c}), pd.DataFrame({'bigrams': most_common_bigrams_1c}), pd.DataFrame({'trigrams': most_common_trigrams_1c}), pd.DataFrame({'fourgrams': most_common_fourgrams_1c})], axis = 1)
df_ngrams = pd.concat([pd.DataFrame({'unigrams': most_common}), pd.DataFrame({'bigrams': most_common_bigrams}), pd.DataFrame({'trigrams': most_common_trigrams}), pd.DataFrame({'fourgrams': most_common_fourgrams})], axis = 1)

#df_ngrams_1c.to_excel('ngrams_1c.xlsx')
df_ngrams.to_excel('ngrams.xlsx')

In [106]:
def split_skill_amount(most_common):
    ngram = []
    ngram_amount = []
    for one in most_common:
        ngram.append(str(one[0]))
        ngram_amount.append(one[1])
    return ngram, ngram_amount

In [578]:
uni, uni_amount = split_skill_amount(most_common)
bi, bi_amount = split_skill_amount(most_common_bigrams)
tri, tri_amount = split_skill_amount(most_common_trigrams)
four, four_amount = split_skill_amount(most_common_fourgrams)

In [579]:
unigrams = pd.DataFrame({'name': uni, 'amount': uni_amount})
bigrams = pd.DataFrame({'name': bi, 'amount': bi_amount})
trigrams = pd.DataFrame({'name': tri, 'amount': tri_amount})
fourgrams = pd.DataFrame({'name': four, 'amount': four_amount})

считать количество во всех n+1 граммах

In [584]:
for one in trigrams['name']:
    if fourgrams['name'].str.contains(one).any() == True:
        ind = trigrams.index[trigrams['name'] == one].tolist()[0]
        trigrams['amount'][ind] = trigrams['amount'][ind] - fourgrams[fourgrams['name'].str.contains(one)]['amount'].sum()
        if trigrams['amount'][ind] < 0:
            trigrams = trigrams[trigrams.name != one]

        #if trigrams[trigrams['name'] == one]['amount'].item() <= (fourgrams[fourgrams['name'].str.contains(one)]['amount'].sum()):
            #trigrams = trigrams[trigrams.name != one]
            #pass

for one in bigrams['name']:
    if trigrams['name'].str.contains(one).any() == True:
        ind = bigrams.index[bigrams['name'] == one].tolist()[0]
        bigrams['amount'][ind] = bigrams['amount'][ind] - (trigrams[trigrams['name'].str.contains(one)]['amount'].sum() + 
                                                           fourgrams[fourgrams['name'].str.contains(one)]['amount'].sum())
        if bigrams['amount'][ind] < 0:
            bigrams = bigrams[bigrams.name != one]
        # if bigrams[bigrams['name'] == one]['amount'].item() <= (trigrams[trigrams['name'].str.contains(one)]['amount'].sum() +
        #                                                         fourgrams[fourgrams['name'].str.contains(one)]['amount'].sum()):
        #     bigrams = bigrams[bigrams.name != one]

for one in unigrams['name']:
    search = one
    if one == 'c++':
         one = 'c+'
    if bigrams['name'].str.contains("'" + one + "'").any() == True:
        ind = unigrams.index[unigrams['name'] == search].tolist()[0]
        unigrams['amount'][ind] = unigrams['amount'][ind] - (bigrams[bigrams['name'].str.contains(one)]['amount'].sum() + 
                                                             trigrams[trigrams['name'].str.contains(one)]['amount'].sum() + 
                                                             fourgrams[fourgrams['name'].str.contains(one)]['amount'].sum())
        if unigrams['amount'][ind] < 0:
            unigrams = unigrams[unigrams.name != search]

        # if unigrams[unigrams['name'] == search]['amount'].item() <= (bigrams[bigrams['name'].str.contains(one)]['amount'].sum() + 
        #                                                              trigrams[trigrams['name'].str.contains(one)]['amount'].sum() +
        #                                                              fourgrams[fourgrams['name'].str.contains(one)]['amount'].sum()):
        #     unigrams = unigrams[unigrams.name != search]

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6624\258165321.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if fourgrams['name'].str.contains(one).any() == True:
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6624\258165321.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  trigrams['amount'][ind] = trigrams['amount'][ind] - fourgrams[fourgrams['name'].str.contains(one)]['amount'].sum()
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_6624\258165321.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trigrams['amount'][ind] = trigrams['amount'][ind] - fourgrams[fourgrams['name'].str.contains(one)]['amount'].sum()
C:\Users\Lenovo

error: multiple repeat at position 12

In [436]:
# print(len(unigrams), len(bigrams), len(trigrams), len(fourgrams))
# trigrams = clean_ngrams(n1gramm = trigrams, n2gram = fourgrams)
# bigrams = clean_ngrams(n1gramm = bigrams, n2gram = trigrams)
# unigrams = clean_ngrams(n1gramm = unigrams, n2gram = bigrams)
print(len(unigrams), len(bigrams), len(trigrams), len(fourgrams))

875 674 183 31


In [473]:
skills_count = unigrams.amount.sum() + bigrams.amount.sum() + trigrams.amount.sum() + fourgrams.amount.sum()
print(skills_count)

105649


Удаление "не навыков"

In [597]:
to_delete_from_dict = pd.read_csv('words_to_delete_from_dict.csv')['0'].to_list()

In [598]:
ngrams_full = pd.concat([unigrams, bigrams, trigrams, fourgrams], axis = 0, ignore_index = True)

In [599]:
ngrams_full['name'] = ngrams_full['name'].apply(lambda x: None if x in to_delete_from_dict else x)

In [600]:
ngrams_full.dropna(inplace = True)

In [601]:
# ngrams_full = pd.concat([unigrams, bigrams, trigrams, fourgrams], axis = 0, ignore_index = True)
ngrams_full.sort_values(by = 'amount', ascending = False, inplace = True, ignore_index = True)
ngrams_full['cumperc'] = ngrams_full['amount'].cumsum()/ngrams_full['amount'].sum()*100

In [602]:
ngrams_full = ngrams_full[ngrams_full['cumperc'] < 80]

In [603]:
ngrams_full

,name,amount,cumperc
0,1с,5896,4.318813
1,sql,5649,8.456698
2,"('аналитический', 'мышление')",3465,10.994807
3,bpmn,2405,12.756466
4,"('бизнес', 'анализ')",2395,14.510801
...,...,...,...
305,кредит,55,79.823321
306,"('регламентировать', 'учёт')",54,79.862876
307,"('analytical', 'skills')",54,79.902431
308,"('математический', 'склад', 'ум')",54,79.941986


по-хорошему, стоит перепроверить получившиеся навыки и все равно кое-что убрать вручную

In [542]:
ngrams_full.to_excel('dict-pareto_v8.xlsx')

удаление ',()

In [543]:
ngrams_full['name'] = ngrams_full['name'].apply(lambda x: re.sub("[\'(),]", '', x))

После ручной обработки n-грамм - получаем словарь

In [545]:
ngrams_full['name'].to_excel('automatic generated dict.xlsx')